## Urgent Data 
> take 2.0

In [3]:
%load_ext autoreload
%autoreload 2

In [11]:
from doubledot import *
import pandas as pd
from time import sleep
import os

In [20]:
sf = Salesforce()

Directory 'atms_download' already exists.


In [21]:
### Clean out old data ?
# for table in Salesforce.model_d.keys():
#     sf.delete_sf_objects(table)

In [22]:
pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167 ]

In [23]:
# put data into atms_downloads
for obj in ['sales', 'contacts', 'memberships']:
    # does this not get written to file?, no. it does not. and we don't want it to because we're working on proccessing rn.
    sf.atms.fetch_data_by_contactIds(obj, pelton_ids) 

Directory 'atms_download' already exists.
my id is 14tzypka
144164
we have contact_id 144164 and obj is sales
http://crm-api-telus.atmsplus.com/api/sales/contact/144164
[{'saleKey': '310928', 'saleAmount': '123.0500', 'paymentAmount': '123.0500', 'saleDate': '2005-08-19T10:23:33.03', 'active': True, 'terminalKey': 4, 'ticketCount': 0, 'eventDate': None, 'booking': {'bookingId': 48112, 'bookingContactKey': 192717, 'bookingContactType': 'Primary     ', 'contactKey': 144164, 'contactIndividualKey': 137810, 'contactOrganizationKey': 5649, 'displayName': 'Pelton, Donna', 'firstName': 'Donna', 'lastName': 'Pelton', 'email': 'tdpelton@shaw.ca', 'phone': '7804592956'}, 'saleComment': None, 'saleDetails': [{'saleDetailId': 760379, 'saleId': 310928, 'itemKey': 7, 'scheduleKey': None, 'rateKey': 3, 'categoryKey': None, 'itemQuantity': 1, 'pricingPriceKey': 207, 'itemPrice': 105.0, 'itemTotal': 105.0, 'couponTotal': 0.0, 'discountTotal': 0.0, 'total': 123.05, 'revenueDate': '2005-08-19T00:00:00', 

In [47]:
sf.atms.obj_d

<ATMS {'contacts': [{'contactId': 144164, 'contactType': 'Individual', 'username': None, 'createdDate': '2006-08-03T16:08:32.313', 'updateDate': '2006-08-03T16:08:32.313', 'emailOptIn': 'N', 'mailOptIn': 'N', 'phoneOptIn': 'N', 'contactIndividualKey': 137810, 'firstName': 'Donna', 'lastName': 'Pelton', 'middleName': None, 'birthDate': '0001-01-01T00:00:00', 'contactOrganizationKey': 0, 'organizationName': None, 'addresses': [{'addressId': 119705, 'contactKey': 144164, 'addressType': 'Home', 'line1': '96 Aspen Crescent', 'line2': None, 'line3': None, 'country': 'Canada', 'province': 'AB', 'county': None, 'city': 'St. Albert', 'postalZipCode': 'T8N 2L8   '}], 'alerts': [], 'contactTypes': [], 'emails': [{'emailId': 8052, 'contactKey': 144164, 'emailType': 'E-Mail', 'address': 'tdpelton@shaw.ca'}], 'faxes': [], 'phones': [{'phoneId': 254342, 'contactKey': 144164, 'phoneType': 'Business', 'phoneNumber': '7804273658', 'extension': None}, {'phoneId': 254343, 'contactKey': 144164, 'phoneType'

In [36]:
sf.atms.write_data_to_json_files()


File 'contacts.json' written successfully.
File 'memberships.json' written successfully.
File 'sales.json' written successfully.
File 'items.json' written successfully.


In [37]:
def remove_duplicates(file_path : str, idx: str = None):
    df = pd.read_csv('sf_download/'+file_path, sep='\t').drop_duplicates(subset=idx)
    df.to_csv('sf_download/'+file_path+'2',sep='\t', index=False)
    return df


In [38]:
def has_duplicates(file_s: str, idx: str = None):
    df = pd.read_csv('sf_download/'+file_s, sep='\t')
    return (df.shape)[0] != (df.drop_duplicates( subset=idx).shape)[0]

## generate upload files

In [98]:
# generate csv files for each object

# obj_l = ['Membership__c','MembershipTerm__c', 'MembershipMember__c', 'Sale__c', 'SaleDetail__c', 'Ticket__c', 'Contact']
obj_l = Salesforce.model_d.keys()
for obj in obj_l:
    # write atms dictionaries to csv file
    sf.process_objects(obj)
    remove_duplicates(obj+'.csv', Salesforce.model_d[obj]['external_id'])
    assert has_duplicates(obj+'.csv2') == False, 'duplicates in '+obj
    try:
        os.remove('sf_download/'+obj+'.csv')
        print('removed '+obj+'.csv')
    except:
        print('no '+obj+'.csv')

sleep(20)
for obj in obj_l:
    os.rename('sf_download/'+obj+'.csv2', 'sf_download/'+obj+'.csv')

Salesforce.process_objects: sf_object_s : Contact
process_contacts
Salesforce: Writing 7 'Contact' objects to /Users/josephmann/Documents/Github/doubledot/sf_download/Contact.csv
removed Contact.csv
Salesforce.process_objects: sf_object_s : Membership__c
Processing memberships data
Salesforce: Writing 11 memberships objects to /Users/josephmann/Documents/Github/doubledot/sf_download/Membership__c.csv
Salesforce: Writing 56 membership_terms objects to /Users/josephmann/Documents/Github/doubledot/sf_download/MembershipTerm__c.csv
Salesforce: Writing 108 membership_members objects to /Users/josephmann/Documents/Github/doubledot/sf_download/MembershipMember__c.csv
removed Membership__c.csv
Salesforce.process_objects: sf_object_s : MembershipTerm__c
Processing memberships data
Salesforce: Writing 11 memberships objects to /Users/josephmann/Documents/Github/doubledot/sf_download/Membership__c.csv
Salesforce: Writing 56 membership_terms objects to /Users/josephmann/Documents/Github/doubledot/

## working from top to here -

> currently Saleforce choking (Saturday morning 11:15 am 2023-05-20)
> still choking at 15:10 pm 2023-05-20

## Perfect Data
> perfect data - no duplicates, every external id is unique, every lookup has a corresponding record

In [ ]:
# externalId_d = {'Contact':'External_Id__c',
#                 'Membership__c':'membershipId__c', 
#         'MembershipTerm__c': 'membershipTermId__c',
#          'MembershipMember__c': 'membershipMemberId__c', 
#         'Sale__c': 'saleKey__c', 
#         'SaleDetail__c':'saleDetailId__c',
#         'Ticket__c':'ticketKey__c',
#           }
# clean SF ?? or upsert - will try upsert

## upload all data to SF
for obj,relations in sf.model_d.items():
    print(obj, relations['external_id'] )
    sf.execute_job(obj, 'upsert', external_id=relations['external_id'], use_ATMS_data=False) 
    sleep(2)

In [ ]:
# """SaleDetail__c saleDetailId__c
# execute_job
# Created job 7508Y00000nHXnfQAG for SaleDetail__c with operation upsert
# Uploading job 7508Y00000nHXnfQAG of object SaleDetail__c
# job status: InProgress
# waiting for job to complete, try 0, status: InProgress
# Failed results:
# "sf__Id"	"sf__Error"	saleDetailId__c	itemKey__c	scheduleKey__c	rateKey__c	categoryKey__c	itemCategory__c	pricingPriceKey__c	itemPrice__c	itemTotal__c	couponTotal__c	discountTotal__c	total__c	revenueDate__c	refundReason__c	refundReasonKey__c	systemPriceOverride__c	MembershipTermKey__r.MembershipTermId__c	saleId__r.saleId__c
# ""	"INVALID_FIELD:Foreign key external ID: 310928 not found for field saleId__c in entity Sale__c:--"	"760379.0"	"7.0"	""	"3.0"	""	""	"207.0"	"105.0"	"105.0"	""	""	"123.05"	"2005-08-19"	""	""	"N"	""	"310928"
# ""	"INVALID_FIELD:Foreign key external ID: 75 not found for field saleId__c in entity Sale__c:--"	"199.0"	"30.0"	"119.0"	"1.0"	"2.0"	""	"163.0"	"115.0"	"115.0"	""	"23.0"	"92.0"	"2001-07-03"	""	""	"N"	""	"75"
# ""	"INVALID_FIELD:Foreign key ex"""

## Generate Dictionary of SF Dataframes

In [100]:
# create a dictionary of dataframes for all SF objects  
df_d = {}
for i in Salesforce.model_d.keys(): 
    df_d[i] = pd.read_csv('sf_download/'+i+'.csv', sep='\t')

In [61]:
df_d['SaleDetail__c'].columns

Index(['saleDetailId__c', 'itemKey__c', 'scheduleKey__c', 'rateKey__c',
       'categoryKey__c', 'itemCategory__c', 'pricingPriceKey__c',
       'itemPrice__c', 'itemTotal__c', 'couponTotal__c', 'discountTotal__c',
       'total__c', 'revenueDate__c', 'refundReason__c', 'refundReasonKey__c',
       'systemPriceOverride__c', 'MembershipTermKey__r.MembershipTermId__c',
       'saleId__r.saleId__c'],
      dtype='object')

In [62]:
## so, am I missing the right look up column??  no. model_d is wrong.
## how does this table get generated? 
df_d['Membership__c'].columns

Index(['membershipId__c', 'memberSince__c', 'updateDate__c'], dtype='object')

In [ ]:
# df_d['SaleDetail__c']['membershipTermKey__c'] # key error 
df_d['SaleDetail__c']['saleId__r.saleId__c'] # key error 
df_d['SaleDetail__c']

In [88]:
## debug
df_d['Sale__c'].columns

Index(['saleId__c', 'saleAmount__c', 'paymentAmount__c', 'saleDate__c',
       'active__c', 'terminalKey__c', 'booking_bookingId__c',
       'booking_bookingContactKey__c', 'booking_contactKey__r.contactId',
       'booking_contactIndividualKey__c', 'booking_contactOrganizationKey__c',
       'booking_displayName__c', 'booking_firstName__c', 'booking_lastName__c',
       'booking_email__c', 'booking_phone__c'],
      dtype='object')

## Verify Lookups 

In [106]:
### verify all lookup are satisfied

# after fixing SF, must recreate csv files AND df_d

from doubledot.crema_sf import match_df

# in words.. verify that all lookups exist in their parent tables
for obj,relations in sf.model_d.items():
    print(obj)
    for fromField, parent in relations['lookups_d'].items():
        # print(obj, fromField, parent)   
        parentExternalId = Salesforce.model_d[parent]['external_id']
        toColumn = df_d[parent][parentExternalId]

        # combine from field and parent external id to get Salesforce lookup field
        newFromField = fromField[:-1]+'r.'+parentExternalId
        # print('keys: ', df_d[obj].keys())
        # print('key: ', newFromField)
        fromColumn = df_d[obj][newFromField]
        indGood_b = fromColumn.isin(toColumn)
        good_b = indGood_b.sum() == len(indGood_b)
        if not good_b:
            print('bad lookup: ', obj, fromField, parent,len(indGood_b), len(indGood_b) - indGood_b.sum())

        # print("--->", obj, fromField, parent, good_b)   




Contact
Membership__c
MembershipTerm__c
MembershipMember__c
Sale__c
SaleDetail__c
bad lookup:  SaleDetail__c membershipTermKey__c MembershipTerm__c 319 303
Ticket__c


In [ ]:
## remove bad SaleDetail__c rows


In [ ]:
booking_contactKey__r.contactId__c
booking_contactKey__r.contactId__c


In [69]:
dict()

{}

In [67]:
l = ['Membership__c.csv', 'MembershipTerm__c.csv', 'MembershipMember__c.csv']
for i in l:
    _df = pd.read_csv('sf_download/'+i, sep='\t')
    print(_df.shape)
    _df2 = _df.drop_duplicates()
    print(_df2.shape)
    _df2.to_csv('sf_download/'+i + '_', sep='\t', index=False)
    # assert df.shape

(4, 3)
(4, 3)
(24, 10)
(24, 10)
(11, 7)
(11, 7)


In [ ]:
# clean SF ?? or upsert - will try upsert
sf.execute_job('Sale__c', 'upsert', external_id='saleKey__c') 

In [ ]:
# clean SF ?? or upsert - will try upsert
sf.execute_job('SaleDetail__c', 'upsert', external_id='saleDetailId__c') 

In [ ]:
# clean SF ?? or upsert - will try upsert
sf.execute_job('Ticket__c', 'upsert', external_id='ticketKey__c')   

In [ ]:
## put Membership__c into dataframe to explore
df = pd.read_csv('sf_download/Membership__c.csv', sep='\t')  
df.drop_duplicates().to_csv('sf_download/Membership__c.csv', sep='\t')

In [ ]:
# clean SF ?? or upsert - will try upsert
## upsert gave duplicate error - so delete first
### delete first gave duplicate error - so check my upload
sf.delete_sf_objects('Membership__c' )
sf.execute_job('Membership__c', 'upsert', external_id='membershipId__c', use_ATMS_data=False) 

In [ ]:
## put Membership__c into dataframe to explore
df = pd.read_csv('sf_download/MembershipTerm__c.csv', sep='\t')  
df.drop_duplicates().to_csv('sf_download/MembershipTerm__c.csv', sep='\t')

In [ ]:
# clean SF ?? or upsert - will try upsert
sf.delete_sf_objects('MembershipTerm__c' )
sf.execute_job('MembershipTerm__c', 'upsert', external_id='membershipTermId__c', use_ATMS_data=False) 

In [ ]:
## put Membership__c into dataframe to explore
df = pd.read_csv('sf_download/MembershipTerm__c.csv', sep='\t')  
df.membershipTermId__c.drop_duplicates().value_counts()

In [ ]:
## put Membership__c into dataframe to explore
col_s = "membershipMemberId__c	membershipTermKey__r.membershipTermId__c	cardNumber__c	membershipNumber__c	cardStatus__c	contactKey__r.External_Id__c	displayName__c"
col_l = col_s.split()

df = pd.read_csv('sf_download/MembershipMember__c.csv', sep='\t')[col_l]
df.drop_duplicates().to_csv('sf_download/MembershipMember__c.csv', sep='\t', index=False)
df

In [ ]:
# clean SF ?? or upsert - will try upsert
sf.execute_job('MembershipMember__c', 'upsert', external_id='MembershipMemberId__c', use_ATMS_data=False) 

In [ ]:
## put Membership__c into dataframe to explore
df = pd.read_csv('sf_download/MembershipMember__c.csv', sep='\t')  
df.membershipMemberId__c.value_counts()